## Packages

In [1]:
import pandas as pd             # Traitement des tableaux de données 

# Chargement des données

Dans un premier temps, l'objectif est de récupérer toutes les valeurs de 2012 à 2022. Les données de 2010 à 2011 (inclus) comprennent des données obsolètes et peu pertinentes pour la construction du modèle. Nous éliminons donc les données des deux premières années.

Dans un second temps, il y a 4 fichiers par an. Il faudra donc récupérer les données et les "merge" afin de construire le jeu de données d'une seule année contenant toutes les informations relatives aux accidents. Cela se traduit par une relation entre le **"*Num_Acc*"** et pour certains fichiers le **"*num_veh*"**.

Ainsi, nous obtenons un fichier complet contenant toutes les colonnes possibles (par exemple, il y aura une colonne **"*secu*"**, et 3 colonnes **"*secu1*"**, **"*secu2*"** et **"*secu3*"**). Il faudra les traiter plus tard.

In [2]:
files = ['USAGERS', 'VEHICULES', 'CARACTERISTIQUES', 'LIEUX',]
data_path = 'Data/TEST/TEST/'

full_data = pd.DataFrame()

for file_name in files: 
    path_to_file = data_path + file_name + '.csv'
    
    data = pd.read_csv(path_to_file, encoding="latin1", sep=",", low_memory=False)
    
    if file_name == 'USAGERS':
        full_data = pd.concat([full_data, data], axis=1)
    else:
        if file_name == 'VEHICULES':
            full_data = pd.merge(full_data, data, on=['Num_Acc', 'num_veh'], how='left')
        else:
            full_data = pd.merge(full_data, data, on=['Num_Acc'], how='left')

In [3]:
full_data.head()

,Num_Acc,place,catu,sexe,trajet,secu,locp,actp,etatp,an_nais,...,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1,vma
0,201200049538,1.0,1,2,1.0,11.0,0.0,0.0,0.0,1954.0,...,0.0,2.0,1.0,15.0,35.0,1.0,1.0,1.0,99.0,NaN
1,201200049538,1.0,1,2,1.0,11.0,0.0,0.0,0.0,1968.0,...,0.0,2.0,1.0,15.0,35.0,1.0,1.0,1.0,99.0,NaN
2,201200049538,1.0,1,2,2.0,11.0,0.0,0.0,0.0,1984.0,...,0.0,2.0,1.0,15.0,35.0,1.0,1.0,1.0,99.0,NaN
3,201200004221,1.0,1,1,0.0,21.0,0.0,0.0,0.0,1973.0,...,0.0,2.0,1.0,0.0,45.0,1.0,0.0,1.0,99.0,NaN
4,201200002457,1.0,1,1,0.0,11.0,0.0,0.0,0.0,1984.0,...,0.0,0.0,1.0,0.0,65.0,1.0,5.0,1.0,99.0,NaN


In [4]:
full_data.to_csv('Data/TEST/TEST/full_test.csv', index=False)

## Traitement des données conformément à celui appliquer sur les données d'entraînement

On applique les mêmes traitements que ceux appliquer au set d'entraînement afin de conserver une homogénéité train/test assurrant au modèle de simplement travailler sur des valeurs inédites et non des formats.

### USAGERS

- **sexe** : manquement de 559 valeurs.
- **an_nais** : manquement de 595 valeurs.
- **trajet** : manquement de 38459 valeurs. <- A TRAITER
- **catu** : manquement de 0 valeur.
- **place** : manquement de 1 valeur.
- **secu1** : manquement de 957 valeurs.
- **secu2** : manquement de 112444 valeurs. <- A TRAITER MAIS PAS ETONNANT
- **secu3** : manquement de 142208 valeurs. <- A TRAITER MAIS PAS ETONNANT

In [5]:
def convert_float_to_int(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)
    
def convert_float_to_cat(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)
    data_frame[col_name] = data_frame[col_name].astype('category')
    
def convert_str_to_cat(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype('category')
    
def convert_str_to_int(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)

### sexe

In [6]:
# Binarisation [0, 1] des valeurs de sexe
full_data['sexe'] = full_data['sexe'].replace({2: 0, -1: pd.NA}).fillna(pd.NA)

possible_values = full_data['sexe'].unique()
na_count = full_data['sexe'].isna().sum() + (full_data['sexe'] == -1).sum()
obs_count = full_data.shape[0]
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes/incorrectes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [0 1 <NA>].
Le nombre de valeurs manquantes/incorrectes est de : 559 pour un total de 142422 observations soit 0.3924955414191628%.


### an_nais

In [7]:
full_data['an_nais'] = full_data['an_nais'].fillna(pd.NA)

possible_values = full_data['an_nais'].unique()
na_count = full_data['an_nais'].isna().sum()
obs_count = full_data.shape[0]
rep_na = na_count / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values}.")
print(f"Le nombre de valeurs manquantes est de : {na_count} pour un total de {obs_count} observations soit {rep_na}%.")

Les valeurs possibles sont : [1954. 1968. 1984. 1973. 1957. 1965. 1989. 1993. 1986. 1971. 1955. 1987.
 1960. 1952. 1981. 1945. 1949. 1924. 1951. 1947. 1948. 1959. 1982. 1974.
 2008. 1978. 1988. 1963. 1990. 1950. 1935. 1933. 1943. 1964. 1977. 1980.
 1998. 1936. 2002. 1985. 1995. 1975. 1969. 1961. 1983. 1946. 1966. 1967.
 2000. 1979. 1953. 1972. 1932. 1992. 1942. 1930. 1944. 1938. 1926. 1970.
 1991. 1920. 1958. 1994. 1996. 1999. 1997. 1956. 2001. 1940. 1937. 1962.
 2009. 1976. 2007. 2003. 1934. 1922. 2011. 2012. 1931. 1941. 2004. 1928.
 2006. 2005. 2010. 1921. 1939. 1927. 1929. 1910. 1919. 1925.   nan 1923.
 1918. 1917. 2013. 1914. 2014. 1915. 2015. 2016. 1911. 2017. 2018. 2019.
 1900. 1901. 2020. 2021. 2022.].
Le nombre de valeurs manquantes est de : 595 pour un total de 142422 observations soit 0.41777253514204266%.


### trajet

In [8]:
# Harmonise les données [1, 2, 3, 4, 5, 9]
full_data['trajet'] = full_data['trajet'].replace([-1, 0], pd.NA).fillna(pd.NA)

possible_values_grav = full_data['trajet'].unique()
na_count_grav = full_data['trajet'].isna().sum()
obs_count = full_data.shape[0]
rep_na_grav = na_count_grav / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values_grav}.")
print(f"Le nombre de valeurs manquantes est de : {na_count_grav} pour un total de {obs_count} observations soit {rep_na_grav}%.")

Les valeurs possibles sont : [1.0 2.0 <NA> 9.0 5.0 4.0 3.0].
Le nombre de valeurs manquantes est de : 38459 pour un total de 142422 observations soit 27.003552821895493%.


### locp

In [9]:
# On peut drop la colonne secu qui ne vaut plus rien
full_data.drop(labels=['locp'], axis=1, inplace=True)

### etatp

In [10]:
# On peut drop la colonne secu qui ne vaut plus rien
full_data.drop(labels=['etatp'], axis=1, inplace=True)

### catu

In [11]:
# Pour les années antérieures à 2019 : catu = 4 -> catv = 99 & catu = 3
full_data.loc[full_data['catu'] == 4, 'catv'] = 99
full_data['catu'] = full_data['catu'].replace({4: 3})

possible_values_grav = full_data['catu'].unique()
na_count_grav = full_data['catu'].isna().sum()
obs_count = full_data.shape[0]
rep_na_grav = na_count_grav / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values_grav}.")
print(f"Le nombre de valeurs manquantes est de : {na_count_grav} pour un total de {obs_count} observations soit {rep_na_grav}%.")

Les valeurs possibles sont : [1 2 3].
Le nombre de valeurs manquantes est de : 0 pour un total de 142422 observations soit 0.0%.


### place

In [12]:
# Si c'est un piéton alors sa place est 10 (gestion des différentes réglementations)
full_data.loc[full_data['catu'] == 3, 'place'] = 10
full_data['place'] = full_data['place'].replace({-1: pd.NA})

possible_values_grav = full_data['place'].unique()
na_count_grav = full_data['place'].isna().sum()
obs_count = full_data.shape[0]
rep_na_grav = na_count_grav / obs_count * 100

print(f"Les valeurs possibles sont : {possible_values_grav}.")
print(f"Le nombre de valeurs manquantes est de : {na_count_grav} pour un total de {obs_count} observations soit {rep_na_grav}%.")

Les valeurs possibles sont : [1.0 2.0 4.0 3.0 10.0 5.0 8.0 7.0 9.0 6.0 <NA>].
Le nombre de valeurs manquantes est de : 1 pour un total de 142422 observations soit 0.0007021387145244414%.


### secu

In [13]:
# Fonction pour extraire le premier chiffre renvoyant à l'équipement utilisé
def extract_first_digit(value):
    if pd.notna(value):
        return int(str(value)[0])
    else:
        return 8

In [14]:
# Récupère les lignes où secu1/secu2/secu3 sont NAN (rare cas après 2019, tous les cas avant 2019)
all_nans = full_data[['secu1', 'secu2', 'secu3']].isna().all(axis=1)
filtered_data = full_data[all_nans]

# Conserve seulement le cas où il y a un second digit = 1, sinon applique la valeur pd.NA (non déterminable)
filtered_data.loc[(filtered_data['secu'] <= 10) | (filtered_data['secu'] % 10 == 2) | (filtered_data['secu'] % 10 == 3), 'secu'] = pd.NA
filtered_data['secu'] = filtered_data['secu'].apply(extract_first_digit)

# Défini dans secu1 le seul équipement noté (1 seul avant 2019) et 8 pour les deux autres
filtered_data['secu1'] = filtered_data['secu']
filtered_data['secu2'] = pd.NA
filtered_data['secu3'] = pd.NA

# Met à jour les colonnes secu/secu1/secu2/secu3
full_data.loc[all_nans, :] = filtered_data

# On peut drop la colonne secu qui ne vaut plus rien
full_data.drop(labels=['secu'], axis=1, inplace=True)

# Dans les cas survenus après 2019
list_secu = ['secu1', 'secu2', 'secu3']

for secu_i in list_secu:
    full_data[secu_i] = full_data[secu_i].replace([-1, 9], pd.NA)

for var in list_secu:
    possible_values_grav = full_data[var].unique()
    na_count_grav = full_data[var].isna().sum()
    obs_count = full_data.shape[0]
    rep_na_grav = na_count_grav / obs_count * 100
    
    print(f"Les valeurs possibles sont : {possible_values_grav}.")
    print(f"Le nombre de valeurs manquantes est de : {na_count_grav} pour un total de {obs_count} observations soit {rep_na_grav}%.")

C:\Users\lucas\AppData\Local\Temp\ipykernel_14924\2762502188.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['secu'] = filtered_data['secu'].apply(extract_first_digit)
C:\Users\lucas\AppData\Local\Temp\ipykernel_14924\2762502188.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['secu1'] = filtered_data['secu']
C:\Users\lucas\AppData\Local\Temp\ipykernel_14924\2762502188.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

Les valeurs possibles sont : [1.0 2.0 8.0 <NA> 4.0 3.0 0.0 6.0 5.0 7.0].
Le nombre de valeurs manquantes est de : 957 pour un total de 142422 observations soit 0.6719467497998904%.
Les valeurs possibles sont : [<NA> 0.0 6.0 8.0 5.0 4.0 1.0 3.0 2.0 7.0].
Le nombre de valeurs manquantes est de : 112444 pour un total de 142422 observations soit 78.9512856159863%.
Les valeurs possibles sont : [<NA> 0.0 6.0 8.0 1.0 5.0 4.0 2.0].
Le nombre de valeurs manquantes est de : 142208 pour un total de 142422 observations soit 99.84974231509177%.


### actp

In [15]:
# On drop la colonne actp qui est trop vide
full_data.drop(labels=['actp'], axis=1, inplace=True)

In [16]:
full_data.to_csv('Data/TEST/TEST/Full_Test_V1.csv', index=False)

In [17]:
full_data.shape

(142422, 54)